<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3

### <center> Автор материала: Кощегулов Эльдар, allincool@mail.ru

## <center> Индивидуальный проект по анализу данных </center>
## <center> Классификация спама в SMS </center>

###  Описание набора данных и признаков

#### Цель работы. 
Задача состоит в том, чтобы построить модель классификации спам сообщений в SMS, на основе имеющихся данных.
#### Входные данные.
Решаться задача будет на датасете взятом тут: https://www.kaggle.com/uciml/sms-spam-collection-dataset

* v1 метка spam/ham
* v2 текст sms

Целевой признак является метка spam/ham является ли SMS спамом или нет

###  Первичный анализ 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [ ]:
df = pd.read_csv('../../data/spam.csv', encoding='latin-1')

In [ ]:
df.head()

In [ ]:
df.info()

#### В заявленных признаках v1 и v2 пропущенных значений нет. Видим что помимо признаков v1 и v2 имеем еще 3 признака. Скорее всего это какой-то мусор

In [ ]:
df['Unnamed: 2'].unique()[: 5]

In [ ]:
df[df['Unnamed: 2'] ==  ' PO Box 5249']

#### Добавим данные из трех "левых" столбцов к тесту SMS и удалим их. Переименуем признаки. Для удобства переобозначим метки
* spam - 1
* ham - 0

In [ ]:
df['v2'] = df['v2'] + df['Unnamed: 2'].fillna('') + df['Unnamed: 3'].fillna('') + df['Unnamed: 4'].fillna('')

In [ ]:
df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace = True)

In [ ]:
df.rename(columns = {'v1' : 'label', 'v2' : 'sms'}, inplace = True)

In [ ]:
df['label'] = df['label'].map({'spam' : 1, 'ham' : 0})

In [ ]:
df.head()

#### Датасет содержит 5572 объекта. Теперь пропущенных значений в нем нет.

In [ ]:
df.info()

#### Посмотрим как выглядит обычное SMS и спам SMS

In [ ]:
df[df['label'] == 0].sample(3)

In [ ]:
df[df['label'] == 1].sample(3)

#### В спам-сообщениях часто много заглавных букв, восклицательных знаков, и чисел, типа поздравляем вы выиграли миллион

#### Посмотрим на распределение классов.

In [ ]:
_, ax = plt.subplots()
plt.bar(np.arange(2), df['label'].value_counts(), color = ['green', 'red'])
ax.set_xticks(np.arange(2))
ax.set_xticklabels(['ham', 'spam']);

In [ ]:
df['label'].value_counts()[1] / df.shape[0], df['label'].value_counts()[0] / df.shape[0]

#### Видим что классы несбалансированы

### Инсайты

#### Здравый смысл подсказывает, что обычно в спам сообщениях вам пишут какие-то левые люди, которые представляются вашими друзьями и зовут куда-то зарегистрироваться или вас поздравляют с выигрышами в лотерею. Значит признаки большого количества заглавных букв, обилия знаков препинания и чисел в текстах сообщений, должны что-то дать

### Генерация признаков

#### Пока что будем генерировать признаки для объединенной выборки. Удалим знаки препинания, удалим опечатки, приведем тексты к нижнему регистру, сгенерируем признаки длина текста, число знаков препинания, наличие символа, не являющегося цифрой или буквой алфавита.

#### Первым признаком который мы создадим будет длина SMS. Обычно SMS имеют ограничения на количество слов, поэтому спамеры чтобы не платить много денежек стараются не превосходить эту длину

In [ ]:
df['len'] = df['sms'].apply(lambda x : len(x.strip().split()))

#### Создадим счетчик знаков препинания в тексте SMS, а затем удалим знаки препинания. В идеале нужен счетчик восклицательных знаков, так как в спаме вас обычно поздравляют с выигрышами в лотереях и прочем и используют много восклицаний

In [ ]:
import regex as re    

In [ ]:
df['punctuation'] = df['sms'].apply(lambda x : len(re.findall("[^\P{P}-]+", x)))
df['punctuation'] = df['sms'].apply(lambda x : len(re.findall("[^\P{P}-]+", x)))

In [ ]:
df['sms'] = df['sms'].apply(lambda x : re.sub("[^\P{P}-]+", "", x))

#### Создадим счетчик заглавных букв в тексте SMS, а затем приведем тексты к нижнему регистру.. Зачастую в спам сообщениях пишут капсом.

In [ ]:
df['capital'] = df['sms'].apply(lambda x : sum(1 for c in x if c.isupper()))

In [ ]:
df['sms'] = df['sms'].apply(lambda x : str.lower(x))

#### Посмотрим какие символы встречаются в текстах. Видим что помимо букв и цифр еще встречается много мусора. Создадим бинарный признак: содержит ли текст SMS символ кроме буквы и цифры. 

In [ ]:
symbols = {}
for x in [item for sublist in list(map(list, df['sms'].tolist())) for item in sublist] :
    if x in symbols :
        symbols[x] += 1
    else :
        symbols[x] = 1
symbols

In [ ]:
volwes = 'aeiou'
consonant = 'bcdfghjklmnpqrstvwxyz'
digits = '0123456789'
alphabet = set(volwes) | set(consonant) | set(digits)

In [ ]:
len(alphabet)

In [ ]:
bad_symbols = [x for x in symbols if x not in alphabet]
bad_symbols = ''.join(set(bad_symbols) - set(' '))
bad_symbols

In [ ]:
df['badsymbol'] = df['sms'].apply(lambda x :1 if len([s for s in x if s in bad_symbols]) > 0 else 0)

#### Попробуем исправить опечатки

In [ ]:
df['sms'] = df['sms'].str.replace('å', 'a').str.replace('ä', 'a').str.replace('â', 'a').str.replace('á', 'a')
df['sms'] = df['sms'].str.replace('õ', 'o').str.replace('ò', 'o').str.replace('ð', 'o').str.replace('ö', '0') \
                    .str.replace('ó', 'o').str.replace('ô', 'o')
df['sms'] = df['sms'].str.replace('û', 'u')
df['sms'] = df['sms'].str.replace('è', 'e')
df['sms'] = df['sms'].str.replace('ì', '1').str.replace('ï', 'l')

#### В спам сообщениях часто упоминаются крупные денежные выигрыши. Нужно создать признаки : наличие числа в тексте и наличие символа валюты

#### Замечаем что среди символов в текстах имеются '$' и '£'. Создадим признак для них.

In [ ]:
df['moneysign'] = df['sms'].apply(lambda x : 1 if ('$' in list(x)) or ('£' in list(x)) else 0 )

####  Остальные символы поудаляем. Вообще, возможно что при удалении знаков препинания мы поудаляли смайлы и возможно наличие/отсутствие смайла будет хорошим признаком. Если будет время надо подумать над этим. Признак исправлял ли я или нет

In [ ]:
symbols = {}
for x in [item for sublist in list(map(list, df['sms'].tolist())) for item in sublist] :
    if x in symbols :
        symbols[x] += 1
    else :
        symbols[x] = 1

In [ ]:
bad_symbols = [x for x in symbols if x not in alphabet]
bad_symbols = ''.join(set(bad_symbols) - set(' '))
bad_symbols

In [ ]:
for symb in bad_symbols : 
    df['sms'] = df['sms'].str.replace(symb, '')

In [ ]:
symbols = {}
for x in [item for sublist in list(map(list, df['sms'].tolist())) for item in sublist] :
    if x in symbols :
        symbols[x] += 1
    else :
        symbols[x] = 1
symbols

In [ ]:
df.head()

#### Создадим признак: наличие в тексте SMS числа(возможно надо проверять не просто число, а число с множествой нулей).

In [ ]:
df['num'] = df['sms'].apply(lambda x : 1 if len([s for s in x if s in digits]) > 0 else 0)

In [ ]:
df.columns

#### Полезность признаков будем исследовать в дальнейшем с помощью модели

#### Разобьем данные на трейн и тест с одинаковым распределением целевой переменной

In [ ]:
target = df['label'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, stratify = target, random_state = 10)

In [ ]:
y_train.sum() / len(y_train), y_test.sum() / len(y_test)

In [ ]:
X_train.shape, X_test.shape

#### В трейне 4457 объектов, в тесте 1115

### Визуальный анализ

#### Построим гистограммы созданных признаков слева и гистограммы созданных признаков в зависимости от целевой переменной справа

In [ ]:
for col in X_train.columns[2 :] :
    fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (20, 10))
#     ax.set_ylabel('% фрагментов', fontsize=12)
#     ax.set_xlabel('Имя автора', fontsize=12) 
    axes[0].set_title(col)
    axes[0].hist(X_train[col], bins = 200);
    axes[1].set_title(col)
    axes[1].hist(X_train[col][X_train['label'] == 0], bins = 200, label = 'ham')
    axes[1].hist(X_train[col][X_train['label'] == 1], bins = 200, label = 'spam')
    plt.show()

#### Судя по гистограммам признаов, почти все спам сообщения содержат символ валюты. Также половина спам сообщений содержит число в своем тексте и опечатку. При генерации этих признаков подобный эффект и ожидался.

In [ ]:
fig, ax = plt.subplots(figsize = (20, 10))
sns.heatmap(X_train[['label', 'len', 'punctuation', 'capital', 'badsymbol',
       'moneysign', 'num']].corr())

#### - Во-первых, длина SMS коррелирует с числом гласных/согласных, числом знаков препинания, тут ничего удивительного.
#### - Во-вторых, видим корреляцию между наличием символа, не являющегося цифрой или буквой алфавита, и наличием символов "$" и "£", так как второе является подмножество первого.
#### - В-третьих, видим корреляцию между целевой переменной и наличием числа в тексте SMS и наличием символа денежки.

### Выбор метрики

#### Решается задача классификации на два класса. Классы несбалансированы, FP - нормальное SMS помечено как спам, это недопустимо. FN - спам помечен как нормальное SMS, допустимо, но не сильно хочется. Поэтому в качестве метрики будем использовать rocauc.

### Выбор модели

#### На заре развития спам-фильтров их строили используя наивный байесовский классификатор, поэтому будем рассматривать эту модель. Также у нас ожидается много признаков после использования преобразования tfidf к тексту SMS, поэтому будем рассматривать логистическую регрессию. 

### Предобработка данных

#### Будем использовать преобразование tfidf для текста SMS, так же отмасштабируем признаки.

In [ ]:
scaler = StandardScaler()

In [ ]:
cols = ['len', 'punctuation', 'capital', 'badsymbol', 'moneysign', 'num']
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train[cols]), columns = cols)
X_test_scaled = pd.DataFrame(scaler.transform(X_test[cols]), columns = cols)

#### Данных у нас не так много, поэтому выбираем кросс-валидацию на 10 фолдов. Для начала посмотрим на наши модели из коробки, ничего не настраивая.

In [ ]:
def valid(model, n, bayes = False) :
    skf = StratifiedKFold(n_splits = n, random_state = 17)
    auc_scores = []
    for train_index, valid_index in skf.split(X_train_scaled, y_train):
        X_train_part, X_valid = X_train_scaled.iloc[train_index], X_train_scaled.iloc[valid_index]
        y_train_part, y_valid = y_train[train_index], y_train[valid_index]
        
        X_train_sms, X_valid_sms = X_train.iloc[train_index]['sms'], X_train.iloc[valid_index]['sms']
        cv = TfidfVectorizer(ngram_range = (1, 3))
        X_train_bow = cv.fit_transform(X_train_sms)
        X_valid_bow = cv.transform(X_valid_sms)     
        if bayes :
            X_train_new = X_train_bow
            X_valid_new = X_valid_bow
        else :
            X_train_new = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_train_bow, X_train_part]))
            X_valid_new = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_valid_bow, X_valid]))
        model.fit(X_train_new, y_train_part)
        model_pred_for_auc = model.predict_proba(X_valid_new)
        auc_scores.append(roc_auc_score(y_valid, model_pred_for_auc[:, 1]))
    return np.mean(auc_scores)

In [ ]:
logit = LogisticRegression(random_state = 17)
bayes = MultinomialNB()

In [ ]:
scores_logit = valid(logit, 10)
print('Logistic regreession - rocauc : {}'.format(scores_logit))

In [ ]:
scores_bayes = valid(bayes, 10, True)
print('Bayessian classfier - rocauc : {}'.format(scores_bayes))

#### Видим, что логистическая регрессия справляется получше. Дальше будем работать только с ней.

### Настройка гиперпараметров и построение кривых валидации и обучения.

In [ ]:
def valid_for_valid_plots(model, n, bayes = False) :
    skf = StratifiedKFold(n_splits = n, random_state = 17)
    auc_scores_cv = []
    auc_scores_valid = []
    for train_index, valid_index in skf.split(X_train_scaled, y_train):
        X_train_part, X_valid = X_train_scaled.iloc[train_index], X_train_scaled.iloc[valid_index]
        y_train_part, y_valid = y_train[train_index], y_train[valid_index]
        
        X_train_sms, X_valid_sms = X_train.iloc[train_index]['sms'], X_train.iloc[valid_index]['sms']
        cv = TfidfVectorizer(ngram_range = (1, 3))
        X_train_bow = cv.fit_transform(X_train_sms)
        X_valid_bow = cv.transform(X_valid_sms)     
        if bayes :
            X_train_new = X_train_bow
            X_valid_new = X_valid_bow
        else :
            X_train_new = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_train_bow, X_train_part]))
            X_valid_new = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_valid_bow, X_valid]))
            
        model.fit(X_train_new, y_train_part)
        auc_scores_cv.append(roc_auc_score(y_train_part, model.predict_proba(X_train_new)[:, 1]))
        model_pred_for_auc = model.predict_proba(X_valid_new)
        auc_scores_valid.append(roc_auc_score(y_valid, model_pred_for_auc[:, 1]))
    return 1 - np.mean(auc_scores_valid), 1 - np.mean(auc_scores_cv)

#### Построим кривые валидации

In [ ]:
Cs = [0.1 * i for i in range(1, 21)]

In [ ]:
scores = []
for c in Cs :
    logit = LogisticRegression(C = c, random_state = 17)
    scores.append(valid_for_valid_plots(logit, 10))

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (20, 10))
plt.plot(Cs, [i[0] for i in scores], color = 'blue', label='holdout')
plt.plot(Cs, [i[1] for i in scores], color = 'red', label='CV')
plt.ylabel("ROCAUC")
plt.xlabel("C")
plt.title('Validation curve for C in (0.1, 2)');


####  Будем перебирать значения C в интервале [0.5, 1.5]. При С < 0.5 происходит недообучение. При С > 1.5 ошибка на трейне упирается в ноль,  а на валидации не падает, это переобучение.

In [ ]:
Cs = np.linspace(0.5, 1.5, 10)

In [ ]:
for c in Cs :
    logit = LogisticRegression(C = c, random_state = 17)
    print(c, valid(logit, 10))

### C_opt = 1.5

In [ ]:
C_opt = 1.5

#### Построим кривые обучения

In [ ]:
def valid_for_train_plots(model, n, alpha, bayes = False) :
    skf = StratifiedKFold(n_splits = n, random_state = 17)
    auc_scores_cv = []
    auc_scores_valid = []
    for train_index, valid_index in skf.split(X_train_scaled[: int(X_train_scaled.shape[0] * alpha)], y_train[: int(X_train_scaled.shape[0] * alpha)]):
        X_train_part, X_valid = X_train_scaled.iloc[train_index], X_train_scaled.iloc[valid_index]
        y_train_part, y_valid = y_train[train_index], y_train[valid_index]
        
        X_train_sms, X_valid_sms = X_train.iloc[train_index]['sms'], X_train.iloc[valid_index]['sms']
        cv = TfidfVectorizer(ngram_range = (1, 3))
        X_train_bow = cv.fit_transform(X_train_sms)
        X_valid_bow = cv.transform(X_valid_sms)     
        if bayes :
            X_train_new = X_train_bow
            X_valid_new = X_valid_bow
        else :
            X_train_new = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_train_bow, X_train_part]))
            X_valid_new = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_valid_bow, X_valid]))
            
        model.fit(X_train_new, y_train_part)
        auc_scores_cv.append(roc_auc_score(y_train_part, model.predict_proba(X_train_new)[:, 1]))
        model_pred_for_auc = model.predict_proba(X_valid_new)
        auc_scores_valid.append(roc_auc_score(y_valid, model_pred_for_auc[:, 1]))
    return np.mean(auc_scores_valid), np.mean(auc_scores_cv)

In [ ]:
alphas = [0.1 * i for i in range(1, 11)]
scores = []
for alpha in  alphas :
    logit = LogisticRegression(C = C_opt, random_state = 17)
    scores.append(valid_for_train_plots(logit, 10, alpha = alpha))

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (20, 10))
plt.plot(alphas, [i[0] for i in scores], color = 'blue', label='holdout')
plt.plot(alphas, [i[1] for i in scores], color = 'red', label='CV')
plt.ylabel("ROCAUC")
plt.xlabel("C")
plt.title('Learnings curve with optimal C');

#### Судя по кривым обучения, происходит недообучение и для улучшения результата надо усложнить модель.

### Прогноз для тестовой выборки

In [ ]:
cv = TfidfVectorizer(ngram_range = (1, 3))
X_train_sms = cv.fit_transform(X_train['sms'])
X_test_sms = cv.transform(X_test['sms'])

In [ ]:
train = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_train_sms, X_train_scaled]))
test = scipy.sparse.csr_matrix(scipy.sparse.hstack([X_test_sms, X_test_scaled]))

In [ ]:
logit = LogisticRegression(C = C_opt, random_state = 17)

In [ ]:
logit.fit(train, y_train)

In [ ]:
for x, y in zip(cols, logit.coef_[0][len(cv.get_feature_names()) :]) :
    print(x, y)

#### Видим, что для нашей модели признаки наличия числа и наличие символа валюты в тексте SMS являются важными, также число слов в тексте и число заглавных букв, а вот признаки наличия опечаток и знаков препинания не так уж и важны.

In [ ]:
logit_pred = logit.predict_proba(test)

In [ ]:
roc_auc_score(y_test, logit_pred[:, 1])

#### Качество на тесте соответствует ожиданиям после кросс-валидации

### Выводы

#### Предложено решение задачи фильтрации спама на основе модели логистической регрессии. Можно использовать подобные спам-фильтры для SMS, электронной почты.

#### Дальнейшее развитие модели может быть связано с лемматизацией/стеммингом текстов SMS. Использовать стекинг/блендинг нескольких моделей.